In [1]:
import requests
from bs4 import BeautifulSoup
import json

In [14]:
url = "http://vitibrasil.cnpuv.embrapa.br/index.php?ano=2023&opcao=opt_04"
response = requests.get(url)

In [15]:
response.text

'<!-- versão 2.00 -->\n<!DOCTYPE html>\n<html lang="pt-BR">\n<head>\n  <meta charset="UTF-8">\n  <meta name="description" content="Na página \'Vitibrasil\' são apresentadas informações referentes à quantidade de uvas processadas, produção e comercialização de vinhos, suco e derivados provenientes do Estado do Rio Grande do Sul, que representa mais de 90% da produção nacional. Também são apresentados os dados de importações e exportações dos produtos da vitivinicultura.">\n  <meta name="keywords" content="Vinhos, Espumante, Uva, Uvas passas, Suco de uva, Produção, Processamento, Comercialização, Importação, Exportação, Publicação" lang="pt-BR">\n  <meta name="author" content="Embrapa Uva e Vinho"> \n  <link href="./figures/favicon.ico" rel="Shortcut Icon" />\n  <link rel="stylesheet" type="text/css" href="vitibrasil.css">\n  <title>Banco de dados de uva, vinho e derivados</title>\n  <script>\n    var x = screen.width;\n    window.onload = function() {  \n      var meta = document.create

In [17]:
soup = BeautifulSoup(response.text, 'html.parser')

In [63]:
tabela = soup.find("table", class_="tb_base tb_dados")
tabela

<table class="tb_base tb_dados">
<thead>
<tr>
<th>Produto </th>
<th>Quantidade (L.) </th>
</tr>
</thead>
<tbody>
<tr>
<td class="tb_item">
                VINHO DE MESA
              </td>
<td class="tb_item">
                187.016.848
              </td>
</tr>
<tr>
<td class="tb_subitem">
                  Tinto
              </td>
<td class="tb_subitem">
                165.097.539
              </td>
</tr>
<tr>
<td class="tb_subitem">
                  Rosado
              </td>
<td class="tb_subitem">
                2.520.748
              </td>
</tr>
<tr>
<td class="tb_subitem">
                  Branco
              </td>
<td class="tb_subitem">
                19.398.561
              </td>
</tr>
<tr>
<td class="tb_item">
                VINHO FINO DE MESA
              </td>
<td class="tb_item">
                18.589.310
              </td>
</tr>
<tr>
<td class="tb_subitem">
                  Tinto
              </td>
<td class="tb_subitem">
                12.450.606
     

In [80]:
all_data = []
item_atual = None

In [81]:
tabela.find_all("tr")

[<tr>
 <th>Produto </th>
 <th>Quantidade (L.) </th>
 </tr>,
 <tr>
 <td class="tb_item">
                 VINHO DE MESA
               </td>
 <td class="tb_item">
                 187.016.848
               </td>
 </tr>,
 <tr>
 <td class="tb_subitem">
                   Tinto
               </td>
 <td class="tb_subitem">
                 165.097.539
               </td>
 </tr>,
 <tr>
 <td class="tb_subitem">
                   Rosado
               </td>
 <td class="tb_subitem">
                 2.520.748
               </td>
 </tr>,
 <tr>
 <td class="tb_subitem">
                   Branco
               </td>
 <td class="tb_subitem">
                 19.398.561
               </td>
 </tr>,
 <tr>
 <td class="tb_item">
                 VINHO FINO DE MESA
               </td>
 <td class="tb_item">
                 18.589.310
               </td>
 </tr>,
 <tr>
 <td class="tb_subitem">
                   Tinto
               </td>
 <td class="tb_subitem">
                 12.450.606
       

In [82]:
tabela_header = [header.text.strip() for header in tabela.find_all("th")]
tabela_header

['Produto', 'Quantidade (L.)']

In [83]:
linhas = tabela.find_all("tr")[1:]
linhas

[<tr>
 <td class="tb_item">
                 VINHO DE MESA
               </td>
 <td class="tb_item">
                 187.016.848
               </td>
 </tr>,
 <tr>
 <td class="tb_subitem">
                   Tinto
               </td>
 <td class="tb_subitem">
                 165.097.539
               </td>
 </tr>,
 <tr>
 <td class="tb_subitem">
                   Rosado
               </td>
 <td class="tb_subitem">
                 2.520.748
               </td>
 </tr>,
 <tr>
 <td class="tb_subitem">
                   Branco
               </td>
 <td class="tb_subitem">
                 19.398.561
               </td>
 </tr>,
 <tr>
 <td class="tb_item">
                 VINHO FINO DE MESA
               </td>
 <td class="tb_item">
                 18.589.310
               </td>
 </tr>,
 <tr>
 <td class="tb_subitem">
                   Tinto
               </td>
 <td class="tb_subitem">
                 12.450.606
               </td>
 </tr>,
 <tr>
 <td class="tb_subitem">
       

In [84]:
for linha in linhas[:-1]:
    td = linha.find("td")
    if td is not None:
        # Verifica se o <td> tem o atributo 'class'
        if td.has_attr("class"):
            # Verifica se a linha é um item principal (tb_item)
            if "tb_item" in td["class"]:
                item_atual = {
                    "produto": td.text.strip(),
                    "quantidade": linha.find_all("td")[1].text.strip().replace(".", "").replace("-", "0"),
                    "subitens": []
                }
                all_data.append(item_atual)
            # Verifica se a linha é um subitem (tb_subitem)
            elif "tb_subitem" in td["class"]:
                subitem = {
                    "produto": td.text.strip(),
                    "quantidade": linha.find_all("td")[1].text.strip().replace(".", "").replace("-", "0")
                }
                if item_atual:
                    item_atual["subitens"].append(subitem)

In [85]:
json_data = json.dumps(all_data, ensure_ascii=False, indent=4)
print(json_data)

[
    {
        "produto": "VINHO DE MESA",
        "quantidade": "187016848",
        "subitens": [
            {
                "produto": "Tinto",
                "quantidade": "165097539"
            },
            {
                "produto": "Rosado",
                "quantidade": "2520748"
            },
            {
                "produto": "Branco",
                "quantidade": "19398561"
            }
        ]
    },
    {
        "produto": "VINHO FINO DE MESA",
        "quantidade": "18589310",
        "subitens": [
            {
                "produto": "Tinto",
                "quantidade": "12450606"
            },
            {
                "produto": "Rosado",
                "quantidade": "1214583"
            },
            {
                "produto": "Branco",
                "quantidade": "4924121"
            }
        ]
    },
    {
        "produto": "VINHO FRIZANTE",
        "quantidade": "2843600",
        "subitens": []
    },
    {
        "produ